In [1]:
import cv2
import numpy as np

In [3]:
# Plage de couleurs de la balle de tennis (jaune/vert)
couleur_min = np.array([20, 100, 100], dtype=np.uint8)
couleur_max = np.array([80, 255, 255], dtype=np.uint8)

# Fonction pour détecter la balle de tennis
def detecter_balle(frame):
    # Conversion de l'image en espace de couleurs HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Extraction des pixels dans la plage de couleurs de la balle de tennis
    masque = cv2.inRange(hsv, couleur_min, couleur_max)

    # Application d'un flou gaussien pour réduire le bruit
    flou = cv2.GaussianBlur(masque, (11, 11), 0)

    # Détection des contours
    contours, _ = cv2.findContours(flou, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Vérifier s'il y a des contours détectés
    if contours:
        # Recherche du plus grand contour correspondant à la balle de tennis
        plus_grand_contour = max(contours, key=cv2.contourArea)

        # Si le contour est suffisamment grand
        if cv2.contourArea(plus_grand_contour) > 100:
            # Récupération du centre et du rayon du cercle
            (x, y), rayon = cv2.minEnclosingCircle(plus_grand_contour)

            # Dessiner le cercle autour de la balle de tennis détectée
            cv2.circle(frame, (int(x), int(y)), int(rayon), (0, 255, 0), 2)

    return frame

# Initialisation de la webcam
capture = cv2.VideoCapture(0)

while True:
    # Capture de la frame de la webcam
    ret, frame = capture.read()

    # Arrêter la boucle si la capture de la frame a échoué
    if not ret:
        break

    # Appel de la fonction de détection de balle
    frame_detectee = detecter_balle(frame)

    # Affichage de la frame détectée
    cv2.imshow("Detection de balle de tennis", frame_detectee)

    # Condition pour quitter la boucle si la touche 'q' est pressée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer la webcam et fermer les fenêtres OpenCV
capture.release()
cv2.destroyAllWindows()